In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Flatten
from keras.utils import np_utils
import scipy
from keras.layers import Dense
import pandas as pd
import glob
import os
import subprocess
from collections import OrderedDict
import torch
import torch.nn as nn
import tensorflow as tf
np.random.seed(5)
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot

In [4]:
dataset = loadtxt('.\\data\\maked_data_100.csv', delimiter=',')
X = dataset[:,0:129]
y = dataset[:,130]
# split into input (X) and output (y) variables

In [5]:
model = Sequential()
model.add(Dense(129, input_dim=129, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X, y, epochs=150, batch_size=10)
# evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/150
20/20 [==============================] - 0s 700us/step - loss: 167.7713 - accuracy: 0.6200
Epoch 2/150
20/20 [==============================] - 0s 650us/step - loss: 45.4153 - accuracy: 0.6900
Epoch 3/150
20/20 [==============================] - 0s 600us/step - loss: 13.4769 - accuracy: 0.8750
Epoch 4/150
20/20 [==============================] - 0s 650us/step - loss: 7.6167 - accuracy: 0.9100
Epoch 5/150
20/20 [==============================] - 0s 600us/step - loss: 5.1397 - accuracy: 0.9200
Epoch 6/150
20/20 [==============================] - 0s 600us/step - loss: 9.6007 - accuracy: 0.8800
Epoch 7/150
20/20 [==============================] - 0s 600us/step - loss: 18.7898 - accuracy: 0.8900
Epoch 8/150
20/20 [==============================] - 0s 600us/step - loss: 5.1644 - accuracy: 0.9300
Epoch 9/150
20/20 [==============================] - 0s 600us/step - loss: 4.2668 - accuracy: 0.9500
Epoch 10/150
20/20 [==============================] - 0s 650us/step - loss: 1.0601 - a

20/20 [==============================] - 0s 600us/step - loss: 6.1784e-08 - accuracy: 1.0000
Epoch 79/150
20/20 [==============================] - 0s 600us/step - loss: 6.0241e-08 - accuracy: 1.0000
Epoch 80/150
20/20 [==============================] - 0s 600us/step - loss: 5.9211e-08 - accuracy: 1.0000
Epoch 81/150
20/20 [==============================] - 0s 650us/step - loss: 5.8255e-08 - accuracy: 1.0000
Epoch 82/150
20/20 [==============================] - 0s 600us/step - loss: 5.7237e-08 - accuracy: 1.0000
Epoch 83/150
20/20 [==============================] - 0s 650us/step - loss: 5.6132e-08 - accuracy: 1.0000
Epoch 84/150
20/20 [==============================] - 0s 600us/step - loss: 5.5300e-08 - accuracy: 1.0000
Epoch 85/150
20/20 [==============================] - 0s 650us/step - loss: 5.4261e-08 - accuracy: 1.0000
Epoch 86/150
20/20 [==============================] - 0s 600us/step - loss: 5.3418e-08 - accuracy: 1.0000
Epoch 87/150
20/20 [==============================] - 0s 60

In [6]:
model.save('.\\data\\test_model\\snorkel_100')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: .\data\test_model\snorkel_100\assets


In [7]:
dataset2 = loadtxt('.\\data\\processed_data_test.csv', delimiter=',')
testX = dataset2[:,0:129]
testY = dataset2[:,130]
result= model.predict(testX)

In [11]:
# generate 2 class dataset
ns_probs = [0 for _ in range(len(testY))]
# fit a model
lr_probs = model.predict(testX)
# keep probabilities for the positive outcome only
# lr_probs = lr_probs[:, 0]
# calculate scores
ns_auc = roc_auc_score(testY, ns_probs)
lr_auc = roc_auc_score(testY, lr_probs)
# summarize scores
print('No Skill: ROC AUC=%.3f' % (ns_auc))
print('Logistic: ROC AUC=%.3f' % (lr_auc))
# calculate roc curves
ns_fpr, ns_tpr, _ = roc_curve(testY, ns_probs)
lr_fpr, lr_tpr, _ = roc_curve(testY, lr_probs)
# plot the roc curve for the model
pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')
# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()

No Skill: ROC AUC=0.500
Logistic: ROC AUC=0.952


<Figure size 640x480 with 1 Axes>

In [13]:
idx  = 0
len(result)
equal_cnt = 0
tp_cnt = 0
tn_cnt = 0
fp_cnt = 0
fn_cnt = 0
for r in result :

    a = r[0]
    b = int(testY[idx])
    #print("r : ", r[0], "a :", a,"b :",b)
    if r[0] >= 0.99 :
        a = 1
    else :
        a = 0
        
    if testY[idx] >= 1 :
        b = 1
    else :
        b = 0
    if a == b :
        equal_cnt += 1
    if a == 1 and b == 1 :
        tp_cnt += 1
    if a == 0 and b == 0 :
        tn_cnt += 1        
    if a == 0 and b == 1 :
        fp_cnt += 1                  
    if a == 1 and b == 0 :
        fn_cnt += 1          
    idx += 1
#print("tp : ",tp_cnt,",tn : ",tn_cnt,",fp : ",fp_cnt,",fn : ",fn_cnt)
precision = tp_cnt / (tp_cnt + fp_cnt)
recall = tp_cnt / (tp_cnt + fn_cnt)
accuracy = (tp_cnt + tn_cnt) / (tp_cnt + fn_cnt + fp_cnt + tn_cnt)
f1_score = 2 * ((precision * recall)/ (precision + recall))
print("tp : ",tp_cnt,",tn : ",tn_cnt,",fp : ",fp_cnt,",fn : ",fn_cnt,",precision : ",precision,",recall : ",recall ,",acc : ", accuracy,",f1_score : ", f1_score)

tp :  342 ,tn :  380 ,fp :  58 ,fn :  20 ,precision :  0.855 ,recall :  0.9447513812154696 ,acc :  0.9025 ,f1_score :  0.8976377952755906
